In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

# Sklearn --> preproccesing 

from sklearn.preprocessing import RobustScaler , PowerTransformer , StandardScaler , MinMaxScaler

# Sklearn --> Models
from sklearn.cluster import KMeans , DBSCAN , AgglomerativeClustering

In [2]:
# Read data
df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
# Check Shape
df.shape

(541909, 8)

In [4]:
# Check About Data Types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
# Check NAN Value
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
# Drop NAN Vlaue
df.dropna(inplace=True)

In [7]:
# Drop columns
df.drop('StockCode' , axis=1 , inplace=True)

In [8]:
# Check Shape After Drop Col
df.shape

(406829, 7)

In [9]:
# Some  Statistical Analysis
df.describe().round(2)

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,406829.00,406829,406829.00,406829.00
mean,12.06,2011-07-10 16:30:57.879207424,3.46,15287.69
min,-80995.00,2010-12-01 08:26:00,0.00,12346.00
25%,2.00,2011-04-06 15:02:00,1.25,13953.00
50%,5.00,2011-07-31 11:48:00,1.95,15152.00
75%,12.00,2011-10-20 13:06:00,3.75,16791.00
max,80995.00,2011-12-09 12:50:00,38970.00,18287.00
std,248.69,NaN,69.32,1713.60


In [10]:
# Some Statistical Analysis For Categorical Features
df.select_dtypes(include='object').describe().T

,count,unique,top,freq
InvoiceNo,406829,22190,576339,542
Description,406829,3896,WHITE HANGING HEART T-LIGHT HOLDER,2070
Country,406829,37,United Kingdom,361878


In [11]:
# Check About Count Of Unique Number
nuniqu = df[['Description' ,'Country' , 'CustomerID']]
for i in nuniqu:
    nunique_var = df[i].nunique()
    print(f'{i} : {nunique_var}')

Description : 3896
Country : 37
CustomerID : 4372


In [12]:
# There Is Some Invoices Contain 'C' stand for Canceled Order 
df['InvoiceNo'].str.contains('C').count()

8905

In [13]:
# Filter df where the 'InvoiceNo' column does not contain the letter 'C'.
df[~df['InvoiceNo'].str.contains('C' , na=False)]

,InvoiceNo,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...
541904,581587,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [15]:
df['Quantity'].unique()

array([     6,      8,      2,     32,      3,      4,     24,     12,
           48,     18,     20,     36,     80,     64,     10,    120,
           96,     23,      5,      1,     -1,     50,     40,    100,
          192,    432,    144,    288,    -12,    -24,     16,      9,
          128,     25,     30,     28,      7,     72,    200,    600,
          480,     -6,     14,     -2,     -4,     -5,     -7,     -3,
           11,     70,    252,     60,    216,    384,     27,    108,
           52,  -9360,     75,    270,     42,    240,     90,    320,
           17,   1824,    204,     69,    -36,   -192,   -144,    160,
         2880,   1400,     19,     39,    -48,    -50,     56,     13,
         1440,     -8,     15,    720,    -20,    156,    324,     41,
          -10,    -72,    -11,    402,    378,    150,    300,     22,
           34,    408,    972,    208,   1008,     26,   1000,    -25,
         1488,    250,   1394,    400,    110,    -14,     37,    -33,
      

In [17]:
# Remove Negative marks  '-' From 'Quantity' Column
df['Quantity'] = df['Quantity'].apply(lambda x: abs(int(x)) if x != '-' else x)

In [18]:
df['Quantity'].unique()

array([    6,     8,     2,    32,     3,     4,    24,    12,    48,
          18,    20,    36,    80,    64,    10,   120,    96,    23,
           5,     1,    50,    40,   100,   192,   432,   144,   288,
          16,     9,   128,    25,    30,    28,     7,    72,   200,
         600,   480,    14,    11,    70,   252,    60,   216,   384,
          27,   108,    52,  9360,    75,   270,    42,   240,    90,
         320,    17,  1824,   204,    69,   160,  2880,  1400,    19,
          39,    56,    13,  1440,    15,   720,   156,   324,    41,
         402,   378,   150,   300,    22,    34,   408,   972,   208,
        1008,    26,  1000,  1488,   250,  1394,   400,   110,    37,
          33,    78,    21,   272,    84,    47,  1728,    38,    53,
          76,   576,    29,  2400,   500,   180,   960,  1296,   147,
         168,   256,    54,    31,   860,  1010,  1356,  1284,   186,
         114,   360,  1930,  2000,  3114,  1300,   670,   176,   648,
          62, 74215,

In [ ]:
# Create New feature Called 'Total Price'
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df.head()

In [ ]:
# 1- What is the most Item has been sold
df.groupby('Description').agg({'Quantity' : 'sum'}).sort_values('Quantity' , ascending=False).head(10)

In [ ]:
# 2- What is the Total Price for each Items
df.groupby('Description').agg({'TotalPrice' : 'sum'}).sort_values('TotalPrice' , ascending=False).head(10)

In [ ]:
# # Create New features Called 'year' & 'month'
df['year'] = df['InvoiceDate'].dt.year
df['month'] = df['InvoiceDate'].dt.month

In [ ]:
df['Country'].value_counts().sort_values(ascending=False).plot(kind='bar' , title='distribution of countries')
plt.show()

In [ ]:

# Based on this Chart this data is colected in the end of 2010
px.pie(data_frame=df , names=df['year'].value_counts().index ,values=df['year'].value_counts().values ,
      hole=0.5 , title= 'Pie chart of year ' )

In [ ]:
# Total of sale for each year. 
df[['year' , 'TotalPrice']].groupby('year')['TotalPrice'].sum().sort_values(ascending=False).plot(kind='bar')
plt.show()

In [ ]:
df.head()

In [ ]:
# # Create New feature Called 'Seasonality'
def Seasonality(value):
    if value in [1,2,3]:
        return 'Q1'
    elif value in [4,5,6]:
        return 'Q2'
    elif value in [7,8,9]:
        return 'Q3'
    else:
        return 'Q4'

df['Seasonality'] = df['month'].apply(Seasonality)
df.head()

In [ ]:
# there is only Quartier year on 2010
df[df['year'] == 2010].groupby('Seasonality')['TotalPrice'].sum().sort_values(ascending=False)

In [ ]:
# Total of sale for each Quartier year on 2011
df[df['year'] == 2011].groupby('Seasonality')['TotalPrice'].sum().sort_values(ascending=False).plot(kind='bar' , 
                                                                        title='Total of sale for each Quartier year on 2011')
plt.show()

In [ ]:
df.groupby('month').agg({'TotalPrice' : 'sum'}).sort_values(by='TotalPrice' , ascending= False)

In [ ]:
df.groupby(['month' , 'year']).agg({'TotalPrice' : 'sum'}).sort_values(by='TotalPrice' , ascending= False).plot(kind= 'bar' ,
                                                                                    title= 'Total price for each month in each year')
plt.show()

# RFM

In [ ]:
df['InvoiceDate'].max()

In [ ]:
today_date = dt.datetime(2011 , 12 ,11 )

In [ ]:
RFM = df.groupby('CustomerID').agg({'InvoiceDate' : lambda date : (today_date - date.max()).days,
                                  'InvoiceNo' : lambda num : num.nunique() ,
                                    'TotalPrice' : lambda TotalPrice : TotalPrice.sum()})
RFM

In [ ]:
RFM.columns = ['Recency' , 'Frequency' , 'Monetary']
RFM

In [ ]:
final_RFM = RFM[RFM['Monetary'] > 0]
final_RFM

In [ ]:
final_RFM['Recency_Score'] = pd.qcut(final_RFM['Recency'] , 5 , labels=[5 , 4 , 3 , 2 , 1])
final_RFM['Frequency_Score'] = pd.qcut(final_RFM['Frequency'].rank(method='first') , 5 , labels=[1,2,3,4,5])
final_RFM['Monetary_Score'] = pd.qcut(final_RFM['Monetary'] , 5 , labels=[1,2,3,4,5])

final_RFM.head()

In [ ]:
final_RFM['RFM_Score'] = (final_RFM['Recency_Score'].astype(str) + final_RFM['Frequency_Score'].astype(str))
final_RFM

In [ ]:
Customer_Segmentation_using_RFM = final_RFM.copy()


In [ ]:
Customer_Segmentation_using_RFM

In [ ]:
seg_map = {
    r'[1-2][1-2]' : 'hibernating',
    r'[1-2][3-4]' : 'at_risk',
    r'[1-2]5' : 'cant_loose' ,
    r'3[1-2]' : 'about_to_sleep' ,
    r'33' : 'need_attention',
    r'[3-4][4-5]' : 'loyal_customers',
    r'41' : 'promising' ,
    r'51' : 'new_customers' ,
    r'[4-5][2-3]' : 'potential_loyalists' ,
    r'5[4-5]' : 'champions'

}

Customer_Segmentation_using_RFM['Segment'] = Customer_Segmentation_using_RFM['RFM_Score'].replace(seg_map , regex=True)
Customer_Segmentation_using_RFM.sample(10)

In [ ]:
Customer_Segmentation_using_RFM[['Segment' , 'Recency' , 'Frequency' , 'Monetary']].groupby('Segment').agg(['mean' , 'count' , 'max']).round(2)

In [ ]:
px.histogram(data_frame= Customer_Segmentation_using_RFM , x = 'Segment' , title='Distribution Of Segment')

# trying clustering using Kmeans 

In [ ]:
final_RFM.reset_index(inplace= True)

In [ ]:
final_RFM['Recency_Score'] = final_RFM['Recency_Score'].astype(int)
final_RFM['Frequency_Score'] = final_RFM['Frequency_Score'].astype(int)
final_RFM['Monetary_Score'] = final_RFM['Monetary_Score'].astype(int)
final_RFM['RFM_Score'] = final_RFM['RFM_Score'].astype(int)

In [ ]:
num_var = final_RFM.select_dtypes(include='number')
fig , axs = plt.subplots(nrows= 3 , ncols = 3 , figsize = (15 , 6))
axs = axs.flatten()

for i , var in enumerate(num_var):
    sns.boxplot(data=final_RFM , y = var , ax = axs[i]  )
    
    
    axs[i].set_title(f"Boxplot of {var}")


fig.tight_layout()

fig.delaxes(axs[8])

plt.show()

In [ ]:
# Outlier handling using IQR 
#def deal_with_outliers(df , value):
    # intial Q1
#    Q1 = df[value].quantile(.25)
    # intial Q3
   # Q3 = df[value].quantile(.75)
    # Define IQR
    #IQR = Q3 - Q1
    # Define Lower value
    #lower = Q1 - 1.5 * IQR
    # Define upper value
   # upper = Q3 + 1.5 * IQR
    # replace values
   # df[value] = np.where(df[value] > upper , upper ,
  #         np.where(df[value] < lower , lower , df[value]))
 #   
#    return df[value]

In [ ]:
#final_RFM['Recency'] = deal_with_outliers(final_RFM , 'Recency')
#final_RFM['Frequency'] = deal_with_outliers(final_RFM , 'Frequency')
#final_RFM['Monetary'] = deal_with_outliers(final_RFM , 'Monetary')

In [ ]:
minmax =  MinMaxScaler()
X_final = minmax.fit_transform(X=final_RFM)
Sta =  StandardScaler()
X_final_Sta = Sta.fit_transform(X=final_RFM)

K_with_iner = {}
for k in range(1 , 14 , 1):
    each_Kmeans = KMeans(n_clusters=k , n_init=10 , random_state=45)
    each_Kmeans.fit(X_final)
    each_inertia = each_Kmeans.inertia_
    K_with_iner[k] = each_inertia

    
keys_K = list(K_with_iner.keys())
vals_inertia = list(K_with_iner.values())

plt.plot(keys_K , vals_inertia , 'bo--')
plt.title('Normalized')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertias')
plt.show()


K_with_iner_Sta = {}
for k in range(1 , 14 , 1):
    each_Kmeans = KMeans(n_clusters=k , n_init=10 , random_state=45)
    each_Kmeans.fit(X_final_Sta)
    each_inertia = each_Kmeans.inertia_
    K_with_iner_Sta[k] = each_inertia


keys_K_Sta = list(K_with_iner_Sta.keys())
vals_inertia_Sta = list(K_with_iner_Sta.values())

plt.plot(keys_K_Sta , vals_inertia_Sta , 'bo--')
plt.title('Standardized')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertias')
plt.show()

In [ ]:
K_Means =  KMeans(n_clusters=4,n_init=10, random_state=45)
Clusters = K_Means.fit_predict(X_final)

In [ ]:
final_df = pd.DataFrame(np.c_[final_RFM ,Clusters] , columns=['CustomerID', 'Recency', 'Frequency', 'Monetary', 'Recency_Score', 'Frequency_Score',
                                                             'Monetary_Score', 'RFM_Score' , 'Clusters'])


In [ ]:
final_df['Clusters'] = final_df['Clusters'] + 1


In [ ]:
final_df.head(20)

In [ ]:
final_df[final_df['Clusters'] == 2].sample(50)